In [21]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import ImageGrab as IG

In [22]:
def grab_screen(bbox=None):
    screenshot = IG.grab(bbox)
    screenshot = np.array(screenshot)
    screenshot = cv2.cvtColor(screenshot,cv2.COLOR_RGB2BGR)
    return screenshot

In [23]:
class Atari : 
    def __init__(self, path):
        image = cv2.imread(path)
        self.image = image
        self.height = image.shape[0]
        self.width = image.shape[1]
        
        self.location = None
    def match(self, screen_img):
        result = cv2.matchTemplate(screen_img, self.image, cv2.TM_CCOEFF_NORMED)
        min_value, max_value, min_loc, max_loc = cv2.minMaxLoc(result)
        top_left = max_loc
        bottom_right = (max_loc[0]+self.width, max_loc[1]+self.height)
        if max_value > 0.85 :
            self.location = (top_left, bottom_right)
            return True
        else :
            self.location = None
            return False


In [24]:
Start = Atari('play.png')
Bars = [Atari('bar.png'), Atari('sidebar.png')]
Ball = Atari('ball2.png')

In [25]:
import time
import pyautogui as pag

In [26]:
while True:
    screen= grab_screen()

    if  Start.match(screen):
        if Start.location!=None:
            top_left_x= int(Start.location[0][0] - 350)
            top_left_y= int( Start.location[1][1] + 16)
            bottom_right_x = int(Start.location[1][0] + 456)
            bottom_right_y = int( Start.location[1][1] + 593)
            Top_left=(top_left_x,top_left_y)
            Bottom_right = (bottom_right_x,bottom_right_y)
            break;  

    

In [27]:
Center_x = int((Start.location[0][0] + Start.location[1][0])/2) 
Center_y = int((Start.location[0][1] + Start.location[1][1])/2)
Center=(Center_x,Center_y)

In [28]:
import pyautogui
pag.leftClick(Center)

In [29]:
while True:
    main_panal = grab_screen(bbox=(Top_left[0], Top_left[1], Bottom_right[0], Bottom_right[1]))
    for Bar in Bars:
        if Bar.match(main_panal):
            if Bar.location:
                cv2.rectangle(main_panal, Bar.location[0], Bar.location[1], (255,0,0), 2)
        
        if Ball.match(main_panal):
            cv2.rectangle(main_panal, Ball.location[0], Ball.location[1],(255,0,0), 2)    
            if Bar.location:
                bar_center_x= (Bar.location[0][0]+Bar.location[1][0])/2
                cursor_x = Ball.location[0][0]- bar_center_x
                cursor_x=min(cursor_x,bottom_right_x-(Bar.width)/2) 

                # if Bar.location[0][1] - Ball.location[1][1] <= 500:
                pag.move(cursor_x,0, 0.5)
        cv2.imshow("Screen", main_panal)
    if cv2.waitKey(1) == ord('q') :
        cv2.destroyAllWindows()
        break
